# Model-1 
GraphSage + Content using default parameters and 3 Layer MLP.

In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl size=489401 sha256=de3c10a5a6768062b7ec5071d9f447d8166dae34865feed793a120b62e119da6
  Stored in directory: /root/.cache/pip/wheels/ef/67/58/6566a3b61c6ec0f2ca0c2c324cd035ef2955601f0fb3197d5f
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.17-cp310-cp310-linux_x86_64.whl size=1078307 sha256=afa43251aca02db1c

In [2]:
from torch_geometric.datasets import UPFD


In [3]:
#defining the train and test split by defining the feature as content and setting the name as Gossipcop
test_data_gos = UPFD(root=".", name="gossipcop", feature="content",split="test")
train_data_gos = UPFD(root=".", name="gossipcop", feature="content", split="train")
val_data_gos = UPFD(root=".", name="gossipcop", feature="content", split="val")
train_data_gos = train_data_gos + val_data_gos


Extracting ./gossipcop/raw/uc
Processing...
Done!


In [4]:
print("Gossipcop Dataset")
print("Train Samples: ", len(train_data_gos))
print("Test Samples: ", len(test_data_gos))

Gossipcop Dataset
Train Samples:  1638
Test Samples:  3826


In [5]:
train_data_gos[0].edge_index


tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          1, 70, 74],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
         55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
         73, 74, 75]])

In [6]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data_gos, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data_gos, batch_size=256, shuffle=False)

In [7]:
import torch
import torch.nn.functional as F
from torch.nn import LeakyReLU, Softmax, Linear, SELU,Dropout
from torch_geometric.nn import SAGEConv, global_max_pool, GATv2Conv, TopKPooling, global_mean_pool
from torch_geometric.transforms import ToUndirected
from torch.nn import LeakyReLU

In [8]:
#defining the GraphSage Model with 3 SageConv layers and 3 unit MLP
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.full1 = Linear(hidden_channels[2],hidden_channels[3])
        self.full2 = Linear(hidden_channels[3],hidden_channels[4])
        self.full3 = Linear(hidden_channels[4],hidden_channels[5])

        self.softmax = Linear(hidden_channels[5],out_channels)

        #droupouts
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)
        self.dp3 = Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h,batch)

        h = self.full1(h).relu()
        h = self.dp1(h)
        h = self.full2(h).relu()
        h = self.dp2(h)
        h = self.full3(h).relu()
        h = self.dp3(h)
        
        h = self.softmax(h)

        return torch.sigmoid(h)

In [9]:
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score 

In [10]:
#specifying number of input features, hidden layer sizes, and number of output channels
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_gos.num_features,[512,512,512,256,256,256],1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #setting optimiser and learning rate as defined by the paper 
lossff = torch.nn.BCELoss()
print(device)

cpu


In [11]:
#defining the train and test function for the model
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1


In [12]:
#printing out the epoch at lowest wloss. 
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  #initialize with a large value

#without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  #update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

#print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.69305 | TestLoss: 0.69284 | TestAcc: 0.49922 | TestF1: 0.00
Epoch: 01 |  TrainLoss: 0.69266 | TestLoss: 0.69221 | TestAcc: 0.49922 | TestF1: 0.00
Epoch: 02 |  TrainLoss: 0.69184 | TestLoss: 0.69106 | TestAcc: 0.49922 | TestF1: 0.00
Epoch: 03 |  TrainLoss: 0.69092 | TestLoss: 0.68902 | TestAcc: 0.51673 | TestF1: 0.07
Epoch: 04 |  TrainLoss: 0.68813 | TestLoss: 0.68485 | TestAcc: 0.79378 | TestF1: 0.83
Epoch: 05 |  TrainLoss: 0.68392 | TestLoss: 0.67598 | TestAcc: 0.90094 | TestF1: 0.90
Epoch: 06 |  TrainLoss: 0.67362 | TestLoss: 0.66354 | TestAcc: 0.69812 | TestF1: 0.57
Epoch: 07 |  TrainLoss: 0.65871 | TestLoss: 0.63625 | TestAcc: 0.87428 | TestF1: 0.89
Epoch: 08 |  TrainLoss: 0.63039 | TestLoss: 0.61618 | TestAcc: 0.72347 | TestF1: 0.62
Epoch: 09 |  TrainLoss: 0.59668 | TestLoss: 0.54759 | TestAcc: 0.81129 | TestF1: 0.77
Epoch: 10 |  TrainLoss: 0.54577 | TestLoss: 0.49868 | TestAcc: 0.81521 | TestF1: 0.84
Epoch: 11 |  TrainLoss: 0.47703 | TestLoss: 0.39494 | 

# Model 2: 
GraphSage + Content with hyperparameters as defined by the paper and 3 Layer MLP

(Embedding size = 128, batch size= 128, l2 Regularization = 0.001)

In [13]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, embedding_size, batch_size, l2_reg_weight):
        super(Net, self).__init__()
        self.embedding_size = embedding_size
        self.batch_size = batch_size
        self.l2_reg_weight = l2_reg_weight
        
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.full1 = Linear(hidden_channels[2], hidden_channels[3])
        self.full2 = Linear(hidden_channels[3], hidden_channels[4])
        self.full3 = Linear(hidden_channels[4], hidden_channels[5])

        self.softmax = Linear(hidden_channels[5], out_channels)

        # Dropouts
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)
        self.dp3 = Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h, batch)

        h = self.full1(h).relu()
        h = self.dp1(h)
        h = self.full2(h).relu()
        h = self.dp2(h)
        h = self.full3(h).relu()
        h = self.dp3(h)
        
        h = self.softmax(h)

        return torch.sigmoid(h)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_gos.num_features,[512,512,512,256,256,256],1, 128, 128, 0.001).to(device) ##setting embedding size=128, batch size=128 and l2 regularization weight as 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
lossff = torch.nn.BCELoss()
print(device)

cpu


In [15]:
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [16]:
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # Initialize with a large value

# Without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  # Update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

# Print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.69325 | TestLoss: 0.69276 | TestAcc: 0.50078 | TestF1: 0.67
Epoch: 01 |  TrainLoss: 0.69297 | TestLoss: 0.69222 | TestAcc: 0.50078 | TestF1: 0.67
Epoch: 02 |  TrainLoss: 0.69206 | TestLoss: 0.69128 | TestAcc: 0.88892 | TestF1: 0.90
Epoch: 03 |  TrainLoss: 0.69082 | TestLoss: 0.68942 | TestAcc: 0.77967 | TestF1: 0.72
Epoch: 04 |  TrainLoss: 0.68892 | TestLoss: 0.68572 | TestAcc: 0.68688 | TestF1: 0.55
Epoch: 05 |  TrainLoss: 0.68445 | TestLoss: 0.67806 | TestAcc: 0.73523 | TestF1: 0.64
Epoch: 06 |  TrainLoss: 0.67336 | TestLoss: 0.66246 | TestAcc: 0.88657 | TestF1: 0.90
Epoch: 07 |  TrainLoss: 0.65732 | TestLoss: 0.65103 | TestAcc: 0.58521 | TestF1: 0.29
Epoch: 08 |  TrainLoss: 0.64153 | TestLoss: 0.61782 | TestAcc: 0.64428 | TestF1: 0.45
Epoch: 09 |  TrainLoss: 0.61380 | TestLoss: 0.55917 | TestAcc: 0.81652 | TestF1: 0.78
Epoch: 10 |  TrainLoss: 0.55531 | TestLoss: 0.50429 | TestAcc: 0.79613 | TestF1: 0.75
Epoch: 11 |  TrainLoss: 0.48437 | TestLoss: 0.40933 | 

# Model 3: 
GraphSage + Content with 2 Layer MLP as defined by the paper. (This is the model implemented in the paper)

In [17]:
import torch
import torch.nn as nn

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, embedding_size, batch_size, l2_reg_weight):
        super(Net, self).__init__()
        self.embedding_size = embedding_size
        self.batch_size = batch_size
        self.l2_reg_weight = l2_reg_weight
        
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.full1 = Linear(hidden_channels[2], hidden_channels[3])
        self.full2 = Linear(hidden_channels[3], hidden_channels[4])
        self.softmax = Linear(hidden_channels[4], out_channels)

        # Dropouts
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h, batch)

        h = self.full1(h).relu()
        h = self.dp1(h)
        h = self.full2(h).relu()
        h = self.dp2(h)
        
        h = self.softmax(h)

        return torch.sigmoid(h)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_gos.num_features,[512,512,512,256,256,256],1, 128, 128, 0.001).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
lossff = torch.nn.BCELoss()
print(device)


cpu


In [19]:
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [21]:
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # Initialize with a large value

# Without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  # Update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

# Print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.00096 | TestLoss: 0.15501 | TestAcc: 0.97648 | TestF1: 0.98
Epoch: 01 |  TrainLoss: 0.00091 | TestLoss: 0.15731 | TestAcc: 0.97595 | TestF1: 0.98
Epoch: 02 |  TrainLoss: 0.00075 | TestLoss: 0.15227 | TestAcc: 0.97674 | TestF1: 0.98
Epoch: 03 |  TrainLoss: 0.00092 | TestLoss: 0.16120 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 04 |  TrainLoss: 0.00109 | TestLoss: 0.15399 | TestAcc: 0.97648 | TestF1: 0.98
Epoch: 05 |  TrainLoss: 0.00074 | TestLoss: 0.15953 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 06 |  TrainLoss: 0.00072 | TestLoss: 0.15307 | TestAcc: 0.97726 | TestF1: 0.98
Epoch: 07 |  TrainLoss: 0.00075 | TestLoss: 0.15550 | TestAcc: 0.97622 | TestF1: 0.98
Epoch: 08 |  TrainLoss: 0.00067 | TestLoss: 0.15654 | TestAcc: 0.97622 | TestF1: 0.98
Epoch: 09 |  TrainLoss: 0.00069 | TestLoss: 0.16039 | TestAcc: 0.97595 | TestF1: 0.98
Epoch: 10 |  TrainLoss: 0.00082 | TestLoss: 0.15843 | TestAcc: 0.97648 | TestF1: 0.98
Epoch: 11 |  TrainLoss: 0.00076 | TestLoss: 0.15594 | 

# Code 4: 
GraphSage + Content with hyperparameters as defined in the paper and replacemenent of 1 MLP layer with 1 RNN Layer:

In [22]:
import torch.nn as nn

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, embedding_size, batch_size, l2_reg_weight):
        super(Net, self).__init__()
        self.embedding_size = embedding_size
        self.batch_size = batch_size
        self.l2_reg_weight = l2_reg_weight
        
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.rnn = nn.RNN(embedding_size, hidden_channels[3], batch_first=True)
        self.full1 = nn.Linear(hidden_channels[3], hidden_channels[4])
        self.softmax = nn.Linear(hidden_channels[4], out_channels)

        # Dropouts
        self.dp1 = nn.Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h, batch)
        
        # Reshape the input tensor for RNN
        h = h.unsqueeze(0)  # Add a time dimension
        h = self.dp1(h)
        
        # Apply RNN
        h, _ = self.rnn(h)
        h = h.squeeze(0)  # Remove the time dimension
        
        h = self.full1(h).relu()
        h = self.softmax(h)

        return torch.sigmoid(h)

In [23]:
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [24]:
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # Initialize with a large value

# Without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  # Update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

# Print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.00002 | TestLoss: 0.49768 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 01 |  TrainLoss: 0.00001 | TestLoss: 0.49612 | TestAcc: 0.97543 | TestF1: 0.98
Epoch: 02 |  TrainLoss: 0.00001 | TestLoss: 0.49493 | TestAcc: 0.97648 | TestF1: 0.98
Epoch: 03 |  TrainLoss: 0.00001 | TestLoss: 0.49738 | TestAcc: 0.97595 | TestF1: 0.98
Epoch: 04 |  TrainLoss: 0.00001 | TestLoss: 0.52107 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 05 |  TrainLoss: 0.00001 | TestLoss: 0.49754 | TestAcc: 0.97543 | TestF1: 0.98
Epoch: 06 |  TrainLoss: 0.00001 | TestLoss: 0.49657 | TestAcc: 0.97622 | TestF1: 0.98
Epoch: 07 |  TrainLoss: 0.00001 | TestLoss: 0.49817 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 08 |  TrainLoss: 0.00002 | TestLoss: 0.52212 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 09 |  TrainLoss: 0.00002 | TestLoss: 0.49617 | TestAcc: 0.97569 | TestF1: 0.98
Epoch: 10 |  TrainLoss: 0.00001 | TestLoss: 0.49514 | TestAcc: 0.97622 | TestF1: 0.98
Epoch: 11 |  TrainLoss: 0.00001 | TestLoss: 0.49751 | 